# LSTM baseline

from kuto

In [1]:
import os
import sys
import glob
import pickle
import random

import numpy as np
import pandas as pd
import scipy.stats as stats
from pathlib import Path
from glob import glob
from tqdm import tqdm

sys.path.append('../../')
import src.utils as utils
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import pytorch_lightning as pl
# from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import EarlyStopping

import wandb
from pytorch_lightning.loggers import WandbLogger


In [2]:
DATA_DIR = Path("/home/knikaido/work/Indoor-Location-Navigation/data/")
WIFI_DIR = DATA_DIR / 'unified-ds-wifi-and-beacon'
MLFLOW_DIR = DATA_DIR / 'mlflow/mlruns'
OUTPUT_DIR = Path('./output/')
MLFLOW_DIR = DATA_DIR / 'mlflow/mlruns'

## config

In [3]:
configs = {
    'loss':{
        'name': 'MSELoss',
        'params':{}
    },
    'optimizer':{
        'name': 'Adam',
        'params':{
            'lr': 0.001,
        }
    },

    'scheduler':{
        'name': 'ReduceLROnPlateau',
        'params':{
            'factor': 0.1,
            'patience': 3,
        }
    },

    'loader':{
        'train':{
            'batch_size': 512,
            'shuffle': True,
            'num_workers': 4,
        },
        'valid':{
            'batch_size': 512,
            'shuffle': False,
            'num_workers': 4,
        },
        'test':{
            'batch_size': 512,
            'shuffle': False,
            'num_workers': 4,
        }
    }
}

In [4]:
# config
config = configs

# globals variable
SEED = 777
MAX_EPOCHS = 200
N_SPLITS = 5
DEBUG = False
# EXP_MESSAGE = config['globals']['exp_message']

EXP_NAME = 12
IS_SAVE = True

utils.set_seed(SEED)

## read data

In [5]:
wifi_path_list = sorted(glob(str(WIFI_DIR / '*train.csv')))
train_df = []
for path in wifi_path_list:
    train_df.append(pd.read_csv(path))
train_df = pd.concat(train_df).reset_index(drop=True)
train_df = train_df.drop('Unnamed: 0', axis=1)
train_df = train_df.rename(columns={'site':'site_id'})
train_df

,site_id,floor,path,x,y,wifi_bssid_0,wifi_rssi_0,wifi_timegap_0,wifi_bssid_1,wifi_rssi_1,...,beacon_timegap_6,beacon_macaddress_7,beacon_distance_7,beacon_timegap_7,beacon_macaddress_8,beacon_distance_8,beacon_timegap_8,beacon_macaddress_9,beacon_distance_9,beacon_timegap_9
0,5a0546857ecc773753327266,-1,5e15730aa280850006f3d005,230.03738,153.496350,c08ad78a45798cfe176a42b35c7381ae602711c5,-46,434,30f85a5e14351468a6dd13718a9da3b0d7b73685,-46,...,1421,ff9a29cb2dcb4100880d02b305d5691e578e66cf,2.083036,2827,26145e606b575396f4ca1bc439d2a9b37fdc6fa0,2.341936,398,aeb5121f95a613552e00b083ee11cd70be497ab5,3.696872,1088
1,5a0546857ecc773753327266,-1,5e15730aa280850006f3d005,231.40290,158.415150,56bc1d2557bf8225384acb992f4b1993a894db63,-47,1856,25d990fc62eb6d5a42994cd7da0f9b6a70fea57a,-47,...,2562,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,2.629927,1757,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,2.949746,2141,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,2.949746,2664
2,5a0546857ecc773753327266,-1,5e15730aa280850006f3d005,232.46200,164.416730,c0beb6ad539d9bd9333c62866c08c844ac69ab28,-48,2767,56bc1d2557bf8225384acb992f4b1993a894db63,-48,...,1351,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,5.137370,1527,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,5.137370,2667,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,5.718217,2644
3,5a0546857ecc773753327266,-1,5e15730aa280850006f3d005,233.94418,171.414170,bfaebb72653fac35c19b00e7ce484dc2897f18bd,-51,136,bfaebb72653fac35c19b00e7ce484dc2897f18bd,-52,...,100,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,6.356626,148,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,6.356626,220,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,6.356626,324
4,5a0546857ecc773753327266,-1,5e15730b1506f2000638fc29,198.36833,163.520630,24677fe9a6f29ace69792429fd85fa8f3efd0192,-49,1919,e43e8b111c0061d7cba667a23d6f1c0143bc73ac,-50,...,567,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,10.591243,2147,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,11.687424,770,26145e606b575396f4ca1bc439d2a9b37fdc6fa0,12.882066,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75273,5dc8cea7659e181adb076a3f,6,5dd0d97c94e4900006125dd9,249.79349,74.839640,5c93fe6e92a16cbcf2259aea405cafe7c073be6e,-40,1848,5c93fe6e92a16cbcf2259aea405cafe7c073be6e,-43,...,3000,-,-99.000000,3000,-,-99.000000,3000,-,-99.000000,3000
75274,5dc8cea7659e181adb076a3f,6,5dd0d97d878f3300066c750b,249.79349,74.839640,5c93fe6e92a16cbcf2259aea405cafe7c073be6e,-46,1875,24489f6ba05ba6afd52ac54e453fd1a2f1128b1a,-79,...,2915,3873a11805bd7adb86762e806d0f20e56e709e76,2.341936,234,-,-99.000000,3000,-,-99.000000,3000
75275,5dc8cea7659e181adb076a3f,6,5dd0d97d878f3300066c750b,249.43129,76.241234,5c93fe6e92a16cbcf2259aea405cafe7c073be6e,-46,404,5c93fe6e92a16cbcf2259aea405cafe7c073be6e,-46,...,1048,3873a11805bd7adb86762e806d0f20e56e709e76,1.850688,636,3873a11805bd7adb86762e806d0f20e56e709e76,2.341936,2045,-,-99.000000,3000
75276,5dc8cea7659e181adb076a3f,6,5dd0d97d878f3300066c750b,242.54440,72.935265,5c93fe6e92a16cbcf2259aea405cafe7c073be6e,-46,2666,5c93fe6e92a16cbcf2259aea405cafe7c073be6e,-50,...,3000,-,-99.000000,3000,-,-99.000000,3000,-,-99.000000,3000


In [6]:
# train_df = pd.read_csv(WIFI_DIR / 'train_all.csv')
test_df = pd.read_csv(WIFI_DIR / 'test.csv')
test_df = test_df.drop('Unnamed: 0', axis=1)
test_df = test_df.rename(columns={'site':'site_id'})
test_df['site_path_timestamp'] = test_df['site_id'] + '_' + test_df['path'] + '_' + test_df['timestamp'].map('{:013}'.format)
test_df

,timestamp,site_id,floor,path,x,y,wifi_bssid_0,wifi_rssi_0,wifi_timegap_0,wifi_bssid_1,...,beacon_macaddress_7,beacon_distance_7,beacon_timegap_7,beacon_macaddress_8,beacon_distance_8,beacon_timegap_8,beacon_macaddress_9,beacon_distance_9,beacon_timegap_9,site_path_timestamp
0,10,5da1389e4db8ce0c98bd0547,0,00ff0c9a71cc37a2ebdd0f05,0,0,889bfa434d66eed8c386ccbc90f445932c43f8dd,-58,1170,29c7d9e757292e7b2b3d00dc4dae7514531b20b4,...,-,-99.000000,3000,-,-99.000000,3000,-,-99.000000,3000,5da1389e4db8ce0c98bd0547_00ff0c9a71cc37a2ebdd0...
1,4048,5da1389e4db8ce0c98bd0547,0,00ff0c9a71cc37a2ebdd0f05,0,0,11567178cc5ca582a37c4733207c77739e1bf5fd,-56,1000,889bfa434d66eed8c386ccbc90f445932c43f8dd,...,-,-99.000000,3000,-,-99.000000,3000,-,-99.000000,3000,5da1389e4db8ce0c98bd0547_00ff0c9a71cc37a2ebdd0...
2,12526,5da1389e4db8ce0c98bd0547,0,00ff0c9a71cc37a2ebdd0f05,0,0,11567178cc5ca582a37c4733207c77739e1bf5fd,-62,1952,11567178cc5ca582a37c4733207c77739e1bf5fd,...,-,-99.000000,3000,-,-99.000000,3000,-,-99.000000,3000,5da1389e4db8ce0c98bd0547_00ff0c9a71cc37a2ebdd0...
3,25542,5da1389e4db8ce0c98bd0547,0,00ff0c9a71cc37a2ebdd0f05,0,0,98d67fadac518296992afddd24e97a2855af9472,-55,2019,889bfa434d66eed8c386ccbc90f445932c43f8dd,...,-,-99.000000,3000,-,-99.000000,3000,-,-99.000000,3000,5da1389e4db8ce0c98bd0547_00ff0c9a71cc37a2ebdd0...
4,37134,5da1389e4db8ce0c98bd0547,0,00ff0c9a71cc37a2ebdd0f05,0,0,11567178cc5ca582a37c4733207c77739e1bf5fd,-54,1977,11567178cc5ca582a37c4733207c77739e1bf5fd,...,-,-99.000000,3000,-,-99.000000,3000,-,-99.000000,3000,5da1389e4db8ce0c98bd0547_00ff0c9a71cc37a2ebdd0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10128,35117,5a0546857ecc773753327266,0,ffcd9524c80c0fa5bb859eaf,0,0,6c381e7e7d8e984394ce02a3da912acc1b7d294e,-60,2011,6c381e7e7d8e984394ce02a3da912acc1b7d294e,...,-,-99.000000,3000,-,-99.000000,3000,-,-99.000000,3000,5a0546857ecc773753327266_ffcd9524c80c0fa5bb859...
10129,41230,5a0546857ecc773753327266,0,ffcd9524c80c0fa5bb859eaf,0,0,46e733fa58deea74d962874847a529fb4897e655,-59,244,6c381e7e7d8e984394ce02a3da912acc1b7d294e,...,5a95e3ee4af260d25d5c13fcf02760820cb6dbdc,67.382098,1334,-,-99.000000,3000,-,-99.000000,3000,5a0546857ecc773753327266_ffcd9524c80c0fa5bb859...
10130,51634,5a0546857ecc773753327266,0,ffcd9524c80c0fa5bb859eaf,0,0,46e733fa58deea74d962874847a529fb4897e655,-49,1166,46e733fa58deea74d962874847a529fb4897e655,...,-,-99.000000,3000,-,-99.000000,3000,-,-99.000000,3000,5a0546857ecc773753327266_ffcd9524c80c0fa5bb859...
10131,60483,5a0546857ecc773753327266,0,ffcd9524c80c0fa5bb859eaf,0,0,e5c220800a2d5ec83e355c3b1a2d7a141947e95f,-53,1309,6c381e7e7d8e984394ce02a3da912acc1b7d294e,...,216d1cba23183d9f79bdd0ba7e45f1d55e611c6e,18.299263,118,9b91bb4419360103cea4e1f7d434878550956b32,18.299263,2225,9e9913d8d1bf56161d199b1655220e8be5711f8f,18.800756,2213,5a0546857ecc773753327266_ffcd9524c80c0fa5bb859...


In [7]:
sub = pd.read_csv(DATA_DIR/'indoor-location-navigation/sample_submission.csv', index_col=0)
sub

,floor,x,y
site_path_timestamp,,,
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000009017,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000015326,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000018763,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000022328,0,75.0,75.0
...,...,...,...
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000082589,0,75.0,75.0
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000085758,0,75.0,75.0
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000090895,0,75.0,75.0


BSSIDとRSSIは100ずつ存在しているけど全てが必要なわけではないみたい  
ここでは20だけ取り出している。

In [8]:
# training target features
NUM_FEATS = 100
BSSID_FEATS = [f'wifi_bssid_{i}' for i in range(NUM_FEATS)]
RSSI_FEATS  = [f'wifi_rssi_{i}' for i in range(NUM_FEATS)]
TIMEGAP_FEATS  = [f'wifi_timegap_{i}' for i in range(NUM_FEATS)]

In [9]:
# get numbers of bssids to embed them in a layer

# train
wifi_bssids = []
# bssidを列ごとにリストに入れていく
for i in BSSID_FEATS:
    wifi_bssids.extend(train_df.loc[:,i].values.tolist())
wifi_bssids = list(set(wifi_bssids))

train_wifi_bssids_size = len(wifi_bssids)
print(f'BSSID TYPES(train): {train_wifi_bssids_size}')

# test
wifi_bssids_test = []
for i in BSSID_FEATS:
    wifi_bssids_test.extend(test_df.loc[:,i].values.tolist())
wifi_bssids_test = list(set(wifi_bssids_test))

test_wifi_bssids_size = len(wifi_bssids_test)
print(f'BSSID TYPES(test): {test_wifi_bssids_size}')


wifi_bssids.extend(wifi_bssids_test)
wifi_bssids_size = len(wifi_bssids)
print(f'BSSID TYPES(all): {wifi_bssids_size}')


BSSID TYPES(train): 54496
BSSID TYPES(test): 27843
BSSID TYPES(all): 82339


## preprocessing

In [10]:
timegaps = train_df[TIMEGAP_FEATS].values
bssids = train_df[BSSID_FEATS].values
rssis = train_df[RSSI_FEATS].values

In [11]:
ordered_timegaps = []
ordered_bssids = []
ordered_rssis = []

for i in tqdm(range(len(train_df))):
    order = np.argsort(timegaps[i])
    ordered_timegaps.append(timegaps[i][order])
    ordered_bssids.append(bssids[i][order])
    ordered_rssis.append(rssis[i][order])
    
ordered_timegaps = np.array(ordered_timegaps)
ordered_bssids = np.array(ordered_bssids)    
ordered_rssis = np.array(ordered_rssis)

100%|██████████| 75278/75278 [00:00<00:00, 137776.38it/s]


In [12]:
train_df[TIMEGAP_FEATS] = ordered_timegaps
train_df[BSSID_FEATS] = ordered_bssids
train_df[RSSI_FEATS] = ordered_rssis
train_df

,site_id,floor,path,x,y,wifi_bssid_0,wifi_rssi_0,wifi_timegap_0,wifi_bssid_1,wifi_rssi_1,...,beacon_timegap_6,beacon_macaddress_7,beacon_distance_7,beacon_timegap_7,beacon_macaddress_8,beacon_distance_8,beacon_timegap_8,beacon_macaddress_9,beacon_distance_9,beacon_timegap_9
0,5a0546857ecc773753327266,-1,5e15730aa280850006f3d005,230.03738,153.496350,c08ad78a45798cfe176a42b35c7381ae602711c5,-46,434,bfaebb72653fac35c19b00e7ce484dc2897f18bd,-68,...,1421,ff9a29cb2dcb4100880d02b305d5691e578e66cf,2.083036,2827,26145e606b575396f4ca1bc439d2a9b37fdc6fa0,2.341936,398,aeb5121f95a613552e00b083ee11cd70be497ab5,3.696872,1088
1,5a0546857ecc773753327266,-1,5e15730aa280850006f3d005,231.40290,158.415150,b0cb9376f8cd87d3e49443e6c7464fe86ea900e3,-66,74,bfaebb72653fac35c19b00e7ce484dc2897f18bd,-59,...,2562,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,2.629927,1757,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,2.949746,2141,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,2.949746,2664
2,5a0546857ecc773753327266,-1,5e15730aa280850006f3d005,232.46200,164.416730,3ce63428d3759cd84971e7ca5114981b2fd18449,-64,835,5c43215d2d7fa8309c357b0534b4e5fb064e7cfa,-56,...,1351,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,5.137370,1527,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,5.137370,2667,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,5.718217,2644
3,5a0546857ecc773753327266,-1,5e15730aa280850006f3d005,233.94418,171.414170,bfaebb72653fac35c19b00e7ce484dc2897f18bd,-51,136,8e57a3171309c6ddd6d203133878fa10e48b5d97,-67,...,100,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,6.356626,148,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,6.356626,220,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,6.356626,324
4,5a0546857ecc773753327266,-1,5e15730b1506f2000638fc29,198.36833,163.520630,24677fe9a6f29ace69792429fd85fa8f3efd0192,-49,1919,739ddddef59162677eb0a789f4e58475b07248fc,-76,...,567,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,10.591243,2147,7063a8d2f2f49ab0d53883f4eda9923b97f97fab,11.687424,770,26145e606b575396f4ca1bc439d2a9b37fdc6fa0,12.882066,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75273,5dc8cea7659e181adb076a3f,6,5dd0d97c94e4900006125dd9,249.79349,74.839640,7150550f0583994950cd821d38d6d9cf7a554b64,-87,56,05642eab9b5515eeaa54e50b7bfa93abf9594455,-87,...,3000,-,-99.000000,3000,-,-99.000000,3000,-,-99.000000,3000
75274,5dc8cea7659e181adb076a3f,6,5dd0d97d878f3300066c750b,249.79349,74.839640,5c93fe6e92a16cbcf2259aea405cafe7c073be6e,-46,1875,b6d081cf13776bfa566f563f7c25bc51e5f5e0e2,-88,...,2915,3873a11805bd7adb86762e806d0f20e56e709e76,2.341936,234,-,-99.000000,3000,-,-99.000000,3000
75275,5dc8cea7659e181adb076a3f,6,5dd0d97d878f3300066c750b,249.43129,76.241234,5c93fe6e92a16cbcf2259aea405cafe7c073be6e,-46,404,d1f7a64fee4b811b4dd0226bef2be71f01214b00,-85,...,1048,3873a11805bd7adb86762e806d0f20e56e709e76,1.850688,636,3873a11805bd7adb86762e806d0f20e56e709e76,2.341936,2045,-,-99.000000,3000
75276,5dc8cea7659e181adb076a3f,6,5dd0d97d878f3300066c750b,242.54440,72.935265,7150550f0583994950cd821d38d6d9cf7a554b64,-87,792,5964a27e0cb3344b0a18540e6b3120c433971c38,-84,...,3000,-,-99.000000,3000,-,-99.000000,3000,-,-99.000000,3000


In [13]:
beacon_columns = [s for s in list(train_df.columns) if 'beacon_' in s]

In [14]:
# preprocess

le = LabelEncoder()
le.fit(wifi_bssids)
le_site = LabelEncoder()
le_site.fit(train_df['site_id'])

ss = StandardScaler()
ss.fit(train_df.loc[:,RSSI_FEATS])

ss_gap = StandardScaler()
ss_gap.fit(train_df.loc[:,TIMEGAP_FEATS])


def preprocess(input_df, le=le, le_site=le_site, ss=ss):
    output_df = input_df.copy()
    # RSSIの正規化
    output_df.loc[:,RSSI_FEATS] = ss.transform(input_df.loc[:,RSSI_FEATS])
    
    # gapの正規化
    output_df.loc[:,TIMEGAP_FEATS] = ss_gap.transform(input_df.loc[:,TIMEGAP_FEATS])

    # BSSIDのLE(1からふる)
    for i in BSSID_FEATS:
        output_df.loc[:,i] = le.transform(input_df.loc[:,i])
#         output_df.loc[:,i] = output_df.loc[:,i] + 1  # 0からではなく1から番号を振りたいため なぜ？

    # site_idのLE
    output_df.loc[:, 'site_id'] = le_site.transform(input_df.loc[:, 'site_id'])
    
    output_df = output_df.drop(beacon_columns, axis=1)

    # なぜ２重でやる？
#     output_df.loc[:,RSSI_FEATS] = ss.transform(output_df.loc[:,RSSI_FEATS])
    return output_df

train = preprocess(train_df)
test = preprocess(test_df)
train    

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: DataConversionWarning: Data 

,site_id,floor,path,x,y,wifi_bssid_0,wifi_rssi_0,wifi_timegap_0,wifi_bssid_1,wifi_rssi_1,...,wifi_timegap_96,wifi_bssid_97,wifi_rssi_97,wifi_timegap_97,wifi_bssid_98,wifi_rssi_98,wifi_timegap_98,wifi_bssid_99,wifi_rssi_99,wifi_timegap_99
0,0,-1,5e15730aa280850006f3d005,230.03738,153.496350,41663,0.222849,-0.515114,41490,0.132786,...,-0.067173,45919,0.327659,-0.069780,28038,0.364874,-0.072009,5971,0.318894,-0.074546
1,0,-1,5e15730aa280850006f3d005,231.40290,158.415150,38161,0.060209,-1.067310,41490,0.206979,...,-0.705287,46296,0.361087,-0.707686,41300,0.348248,-0.709593,13645,0.414933,-0.711877
2,0,-1,5e15730aa280850006f3d005,232.46200,164.416730,13281,0.076473,0.099971,19970,0.231710,...,0.802258,16806,0.401201,0.799367,10708,0.364874,0.796698,41711,0.428180,0.793818
3,0,-1,5e15730aa280850006f3d005,233.94418,171.414170,41490,0.182189,-0.972210,30748,0.141029,...,-0.547753,13281,0.384487,-0.550203,10708,0.381499,-0.552189,1797,0.335453,-0.554536
4,0,-1,5e15730b1506f2000638fc29,198.36833,163.520630,7849,0.198453,1.762696,25034,0.066836,...,-0.888745,28277,0.371116,-0.891084,27665,0.358224,-0.892898,20567,0.325518,-0.895110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75273,23,6,5dd0d97c94e4900006125dd9,249.79349,74.839640,24515,-0.110562,-1.094920,1146,-0.023845,...,1.266885,0,-2.757797,1.263842,0,-2.740793,1.260938,0,-2.721228,1.257875
75274,23,6,5dd0d97d878f3300066c750b,249.79349,74.839640,20039,0.222849,1.695205,39456,-0.032089,...,1.266885,0,-2.757797,1.263842,0,-2.740793,1.260938,0,-2.721228,1.257875
75275,23,6,5dd0d97d878f3300066c750b,249.43129,76.241234,20039,0.222849,-0.561130,45394,-0.007358,...,1.266885,0,-2.757797,1.263842,0,-2.740793,1.260938,0,-2.721228,1.257875
75276,23,6,5dd0d97d878f3300066c750b,242.54440,72.935265,24515,-0.110562,0.034015,19357,0.000886,...,1.238967,24138,0.307601,1.235934,16364,0.301696,1.233044,47275,0.322206,1.229992


In [15]:
site_count = len(train['site_id'].unique())
site_count

24

## PyTorch model
- embedding layerが重要  

In [16]:
# dataset
from torch.utils.data import Dataset, DataLoader
class IndoorDataset(Dataset):
    def __init__(self, df, phase='train'):
        self.df = df
        self.phase = phase
        self.bssid_feats = df[BSSID_FEATS].values.astype(int)
        self.rssi_feats = df[RSSI_FEATS].values.astype(np.float32)
        self.timegap_feats = df[TIMEGAP_FEATS].values.astype(np.float32)
        self.site_id = df['site_id'].values.astype(int)

        if phase in ['train', 'valid']:
            self.xy = df[['x', 'y']].values.astype(np.float32)
            self.floor = df['floor'].values.astype(np.float32)
        
    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        
        feature = {
            'BSSID_FEATS':self.bssid_feats[idx],
            'RSSI_FEATS':self.rssi_feats[idx],
            'TIMEGAP_FEATS':self.timegap_feats[idx],
            'site_id':self.site_id[idx]
        }
        if self.phase in ['train', 'valid']:
            target = {
                'xy':self.xy[idx],
                'floor':self.floor[idx]
            }
        else:
            target = {}
        return feature, target

In [17]:
import torch
from torch import nn

class LSTMModel(nn.Module):
    def __init__(self, bssid_size=94248, site_size=24, embedding_dim=64):
        super(LSTMModel, self).__init__()
        
        # bssid
        # ->64次元に圧縮後sequence化にする
        # wifi_bssids_sizeが辞書の数を表す
        self.bssid_embedding = nn.Embedding(bssid_size, 64, max_norm=True)
        # site
        # ->2次元に圧縮後sequence化する
        # site_countが辞書の数を表す       
        self.site_embedding = nn.Embedding(site_size, 64, max_norm=True)

        # rssi
        # 次元を64倍に線形変換
        self.rssi = nn.Sequential(
            nn.BatchNorm1d(NUM_FEATS),
            nn.Linear(NUM_FEATS, NUM_FEATS * 64)
        )
        
        self.timegap = nn.Sequential(
            nn.BatchNorm1d(NUM_FEATS),
            nn.Linear(NUM_FEATS, NUM_FEATS * 64)
        )
        
        concat_size = 64 + (NUM_FEATS * 64) + (NUM_FEATS * 64) + (NUM_FEATS * 64)
        self.linear_layer2 = nn.Sequential(
            nn.BatchNorm1d(concat_size),
            nn.Dropout(0.3),
            nn.Linear(concat_size, 256),
            nn.ReLU()
        )
        self.bn1 = nn.BatchNorm1d(concat_size)

        self.flatten = nn.Flatten()

        self.dropout1 = nn.Dropout(0.3)
        self.linear1 = nn.Linear(in_features=concat_size, out_features=256)#, bias=False)
        self.bn2 = nn.BatchNorm1d(256)

        self.batch_norm1 = nn.BatchNorm1d(1)
        self.lstm1 = nn.LSTM(input_size=256,hidden_size=128,dropout=0.3, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=128,hidden_size=16,dropout=0.1, batch_first=True)

        self.fc_xy = nn.Linear(16, 2)
        # self.fc_x = nn.Linear(16, 1)
        # self.fc_y = nn.Linear(16, 1)
        self.fc_floor = nn.Linear(16, 1)

    
    def forward(self, x):
        # input embedding
        batch_size = x["site_id"].shape[0]
        x_bssid = self.bssid_embedding(x['BSSID_FEATS'])
        x_bssid = self.flatten(x_bssid)
        
        x_site_id = self.site_embedding(x['site_id'])
        x_site_id = self.flatten(x_site_id)

        x_rssi = self.rssi(x['RSSI_FEATS'])
        
        x_timegap = self.timegap(x['TIMEGAP_FEATS'])

        x = torch.cat([x_bssid, x_site_id, x_rssi, x_timegap], dim=1)
        x = self.linear_layer2(x)

        # lstm layer
        x = x.view(batch_size, 1, -1)  # [batch, 1]->[batch, 1, 1]
        x = self.batch_norm1(x)
        x, _ = self.lstm1(x)
        x = torch.relu(x)
        x, _ = self.lstm2(x)
        x = torch.relu(x)

        # output [batch, 1, 1] -> [batch]
        # x_ = self.fc_x(x).view(-1)
        # y_ = self.fc_y(x).view(-1)
        xy = self.fc_xy(x).squeeze(1)
        floor = torch.relu(self.fc_floor(x)).view(-1)
        # return {"x":x_, "y":y_, "floor":floor} 
        return {"xy": xy, "floor": floor}

In [18]:
def mean_position_error(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]

def to_np(input):
    return input.detach().cpu().numpy()

In [19]:
def get_optimizer(model: nn.Module, config: dict):
    optimizer_config = config["optimizer"]
    optimizer_name = optimizer_config.get("name")
    base_optimizer_name = optimizer_config.get("base_name")
    optimizer_params = optimizer_config['params']

    if hasattr(optim, optimizer_name):
        optimizer = optim.__getattribute__(optimizer_name)(model.parameters(), **optimizer_params)
        return optimizer
    else:
        base_optimizer = optim.__getattribute__(base_optimizer_name)
        optimizer = globals().get(optimizer_name)(
            model.parameters(), 
            base_optimizer,
            **optimizer_config["params"])
        return  optimizer

def get_scheduler(optimizer, config: dict):
    scheduler_config = config["scheduler"]
    scheduler_name = scheduler_config.get("name")

    if scheduler_name is None:
        return
    else:
        return optim.lr_scheduler.__getattribute__(scheduler_name)(
            optimizer, **scheduler_config["params"])


def get_criterion(config: dict):
    loss_config = config["loss"]
    loss_name = loss_config["name"]
    loss_params = {} if loss_config.get("params") is None else loss_config.get("params")
    if hasattr(nn, loss_name):
        criterion = nn.__getattribute__(loss_name)(**loss_params)
    else:
        criterion = globals().get(loss_name)(**loss_params)

    return criterion

def worker_init_fn(worker_id):                                                          
    np.random.seed(np.random.get_state()[1][0] + worker_id)

In [20]:
# Learner class(pytorch-lighting)
class Learner(pl.LightningModule):
    def __init__(self, model, config):
        super().__init__()
        self.model = model
        self.config = config
        self.xy_criterion = get_criterion(config)
        self.f_criterion = get_criterion(config)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        output = self.model(x)
        loss = self.xy_criterion(output["xy"], y["xy"])
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        output = self.model(x)
        xy_loss = self.xy_criterion(output["xy"], y["xy"])
        f_loss = self.f_criterion(output["floor"], y["floor"])
        loss = xy_loss  # + f_loss
        mpe = mean_position_error(
            to_np(output['xy'][:, 0]), to_np(output['xy'][:, 1]), 0, 
            to_np(y['xy'][:, 0]), to_np(y['xy'][:, 1]), 0)
        
        # floor lossは現状は無視して良い
        self.log(f'Loss/val', loss, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        self.log(f'Loss/xy', xy_loss, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        self.log(f'Loss/floor', f_loss, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        self.log(f'MPE/val', mpe, on_step=False, on_epoch=True, prog_bar=False, logger=True)
        return mpe
    
    def validation_epoch_end(self, outputs):
        avg_loss = np.mean(outputs)
        print(f'epoch = {self.current_epoch}, mpe_loss = {avg_loss}')

    def configure_optimizers(self):
        optimizer = get_optimizer(self.model, self.config)
        scheduler = get_scheduler(optimizer, self.config)
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "Loss/val"}

In [21]:
# oof
def evaluate(model, loaders, phase):
    x_list = []
    y_list = []
    f_list = []
    with torch.no_grad():
        for batch in loaders[phase]:
            x, y = batch
            output = model(x)
            x_list.append(to_np(output['xy'][:, 0]))
            y_list.append(to_np(output['xy'][:, 1]))
            f_list.append(to_np(output['floor']))

    x_list = np.concatenate(x_list)
    y_list = np.concatenate(y_list)
    f_list = np.concatenate(f_list)
    return x_list, y_list, f_list

## train

In [22]:
oofs = []  # 全てのoofをdfで格納する
predictions = []  # 全ての予測値をdfで格納する
val_scores = []
# skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
gkf = GroupKFold(n_splits=N_SPLITS)
# for fold, (trn_idx, val_idx) in enumerate(skf.split(train.loc[:, 'path'], train.loc[:, 'path'])):
for fold, (trn_idx, val_idx) in enumerate(gkf.split(train.loc[:, 'path'], groups=train.loc[:, 'path'])):

    # 指定したfoldのみループを回す

    print('=' * 20)
    print(f'Fold {fold}')
    print('=' * 20)

    # train/valid data
    trn_df = train.loc[trn_idx, BSSID_FEATS + RSSI_FEATS + TIMEGAP_FEATS + ['site_id', 'x','y','floor']].reset_index(drop=True)
    val_df = train.loc[val_idx, BSSID_FEATS + RSSI_FEATS + TIMEGAP_FEATS + ['site_id', 'x','y','floor']].reset_index(drop=True)

    # data loader
    loaders = {}
    loader_config = config["loader"]
    loaders["train"] = DataLoader(IndoorDataset(trn_df, phase="train"), **loader_config["train"], worker_init_fn=worker_init_fn) 
    loaders["valid"] = DataLoader(IndoorDataset(val_df, phase="valid"), **loader_config["valid"], worker_init_fn=worker_init_fn)
    loaders["test"] = DataLoader(IndoorDataset(test, phase="test"), **loader_config["test"], worker_init_fn=worker_init_fn)
    
    # model
    model = LSTMModel(wifi_bssids_size, site_count)
    model_name = model.__class__.__name__
    
    # loggers
    RUN_NAME = f'exp{str(EXP_NAME)}'
    wandb.init(project='Indoor_Location_Navigation', entity='sqrt4kaido', group=RUN_NAME, job_type=RUN_NAME + f'-fold-{fold}')
    wandb.run.name = RUN_NAME + f'-fold-{fold}'
    wandb_config = wandb.config
    wandb_config.model_name = model_name
    wandb.watch(model)
    
    
    loggers = []
    loggers.append(WandbLogger())

    learner = Learner(model, config)
    
    # callbacks
    callbacks = []
    checkpoint_callback = ModelCheckpoint(
        monitor=f'Loss/val',
        mode='min',
        dirpath=OUTPUT_DIR,
        verbose=False,
        filename=f'{model_name}-{learner.current_epoch}-{fold}')
    callbacks.append(checkpoint_callback)

    early_stop_callback = EarlyStopping(
        monitor='Loss/val',
        min_delta=0.00,
        patience=3,
        verbose=True,
        mode='min')
    callbacks.append(early_stop_callback)
    
    trainer = pl.Trainer(
        logger=loggers,
        checkpoint_callback=callbacks,
        max_epochs=MAX_EPOCHS,
        default_root_dir=OUTPUT_DIR,
        gpus=1,
        fast_dev_run=DEBUG,
        deterministic=True,
        benchmark=True,
#         precision=16,
#         progress_bar_refresh_rate=0  # vscodeの時progress barの動作が遅いので表示しない
        )


    trainer.fit(learner, train_dataloader=loaders['train'], val_dataloaders=loaders['valid'])

    #############
    # validation (to make oof)
    #############
    model.eval()
    oof_x, oof_y, oof_f = evaluate(model, loaders, phase="valid")
    val_df["oof_x"] = oof_x
    val_df["oof_y"] = oof_y
    val_df["oof_floor"] = oof_f
    oofs.append(val_df)
    
    val_score = mean_position_error(
        val_df["oof_x"].values, val_df["oof_y"].values, 0,
        val_df['x'].values, val_df['y'].values, 0)
    val_scores.append(val_score)
    print(f"fold {fold}: mean position error {val_score}")

    #############
    # inference
    #############
    preds_x, preds_y, preds_f = evaluate(model, loaders, phase="test")
    test_preds = pd.DataFrame(np.stack((preds_f, preds_x, preds_y))).T
    test_preds.columns = sub.columns
    test_preds["site_path_timestamp"] = test["site_path_timestamp"]
    test_preds["floor"] = test_preds["floor"].astype(int)
    predictions.append(test_preds)
    

Fold 0


/home/user/.local/lib/python3.6/site-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/home/user/.local/lib/python3.6/site-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
wandb: Currently logged in as: sqrt4kaido (use `wandb login --relogin` to force relogin)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type      | Params
-------------------------------------------
0 | model        | LSTMModel | 16.7 M
1 | xy_criterion | MSELoss   | 0     
2 | f_criterion  | MSELoss   | 0     
-------------------------------------------
16.7 M    Trainable params
0         Non-trainable params
16.7 M    Total params
66.851    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

epoch = 0, mpe_loss = 185.5595245361328


...

epoch = 197, mpe_loss = 33.959087824552725


Validating: 0it [00:00, ?it/s]

epoch = 198, mpe_loss = 33.67124149426938


Validating: 0it [00:00, ?it/s]

epoch = 199, mpe_loss = 33.3629050232554
fold 4: mean position error 33.39957943653497


In [23]:
if len(oofs) > 1:
    oofs_df = pd.concat(oofs)
else:
    oofs_df = oofs[0]
oofs_df.to_csv(str(OUTPUT_DIR) + f"/oof{EXP_NAME}.csv", index=False)
oofs_df

,wifi_bssid_0,wifi_bssid_1,wifi_bssid_2,wifi_bssid_3,wifi_bssid_4,wifi_bssid_5,wifi_bssid_6,wifi_bssid_7,wifi_bssid_8,wifi_bssid_9,...,wifi_timegap_97,wifi_timegap_98,wifi_timegap_99,site_id,x,y,floor,oof_x,oof_y,oof_floor
0,41663,41490,6097,13281,7849,17205,40545,38814,28277,40389,...,-0.069780,-0.072009,-0.074546,0,230.03738,153.496350,-1,177.019577,164.333542,0.220378
1,38161,41490,7941,13281,31760,41300,17205,10708,6289,40545,...,-0.707686,-0.709593,-0.711877,0,231.40290,158.415150,-1,177.019119,164.348785,0.220364
2,13281,19970,41490,30748,33953,29390,10708,53418,38161,46296,...,0.799367,0.796698,0.793818,0,232.46200,164.416730,-1,177.035110,163.737869,0.220855
3,41490,30748,24148,38161,29390,44510,6289,45919,16936,8134,...,-0.550203,-0.552189,-0.554536,0,233.94418,171.414170,-1,177.019058,164.354492,0.220360
4,27602,53675,21370,44510,47170,8942,41125,17256,4140,20358,...,-0.805365,-0.807223,-0.809469,0,210.86192,165.376080,-1,177.018921,164.355209,0.220359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15050,20039,46329,54181,39456,54484,16364,44446,15563,39565,33741,...,1.263842,1.260938,1.257875,23,249.43129,76.241234,6,163.109192,118.054108,0.383541
15051,21863,15563,39628,39565,53086,1942,19357,39456,18998,47308,...,1.263842,1.260938,1.257875,23,237.22395,73.177680,6,162.498444,117.116028,0.372976
15052,20039,54181,486,15563,39628,53754,21863,39565,53086,1942,...,1.263842,1.260938,1.257875,23,242.54440,72.935265,6,163.732056,119.006485,0.394242
15053,20039,53086,2282,39628,16364,39565,1942,19357,24138,18998,...,1.263842,1.260938,1.257875,23,249.43129,76.241234,6,163.867401,119.213043,0.396559


In [24]:
    # foldの結果を平均した後、reindexでsubmission fileにindexを合わせる
all_preds = pd.concat(predictions).groupby('site_path_timestamp').mean().reindex(sub.index)

all_preds

,floor,x,y
site_path_timestamp,,,
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,0,96.238533,100.854156
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000009017,0,86.830490,98.938667
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000015326,0,87.715874,102.928299
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000018763,0,86.239601,104.513565
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000022328,0,85.633614,106.807449
...,...,...,...
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000082589,0,162.991730,121.536186
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000085758,0,163.648239,122.691208
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000090895,0,166.087997,126.914223


In [25]:
# floorの数値を置換
simple_accurate_99 = pd.read_csv('../01/submission.csv')
all_preds['floor'] = simple_accurate_99['floor'].values
all_preds

,floor,x,y
site_path_timestamp,,,
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,0,96.238533,100.854156
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000009017,0,86.830490,98.938667
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000015326,0,87.715874,102.928299
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000018763,0,86.239601,104.513565
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000022328,0,85.633614,106.807449
...,...,...,...
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000082589,5,162.991730,121.536186
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000085758,5,163.648239,122.691208
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000090895,5,166.087997,126.914223


In [26]:
all_preds.to_csv(str(OUTPUT_DIR) + f"/sub{EXP_NAME}.csv")

In [27]:
print(f"CV:{np.mean(val_scores)}")

CV:35.91180946326729


In [27]:
wandb.init(project='Indoor_Location_Navigation', entity='sqrt4kaido', group=RUN_NAME, job_type='summary')
wandb.run.name = 'summary'
wandb.log({'CV_score': np.mean(val_scores)})
wandb.save(utils.get_notebook_path())
wandb.finish()

Loss/val,878.90088
Loss/xy,878.90088
Loss/floor,5.63142
MPE/val,26.88436
epoch,199
trainer/global_step,23599
_runtime,384
_timestamp,1617432959
_step,199


Loss/val,██▇▇▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
Loss/xy,██▇▇▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
Loss/floor,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁███▇▇▇▇▇▇▇▆▆▆▄▃▃▃▃▃▃▃▂▂▂▂
MPE/val,██▇▇▇▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


CV_score,34.50177
_runtime,2
_timestamp,1617433227
_step,0


CV_score,▁
_runtime,▁
_timestamp,▁
_step,▁
